# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Setting up libCEED for Python

Clone or download libCEED by running

In [ ]:
! git clone https://github.com/CEED/libCEED.git

Then move to the libCEED folder

In [ ]:
cd libCEED

And compile the base library by running

In [ ]:
! make

Build libCEED for Python by running

In [ ]:
python setup.py build install

Or without superuser permissions

In [ ]:
python setup.py build install --user

## CeedOperator

Here we show some basic examples to illustrate the `CeedOperator` class. In libCEED, a `CeedOperator` defines the finite/spectral element operator associated to a `QFunction` (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

* In the following example, we create and apply a CeedOperator for the mass matrix in 1D.

In [1]:
import libceed
import numpy as np

ceed = libceed.Ceed()

nelem = 15
p = 5
q = 8
nx = nelem + 1
nu = nelem*(p-1) + 1

# Vectors
x = ceed.Vector(nx)
x_array = np.zeros(nx)
for i in range(nx):
  x_array[i] = i / (nx - 1.0)
x.set_array(x_array, cmode=libceed.USE_POINTER)

qdata = ceed.Vector(nelem*q)
u = ceed.Vector(nu)
v = ceed.Vector(nu)

# Restrictions
indx = np.zeros(nx*2, dtype="int32")
for i in range(nx):
  indx[2*i+0] = i
  indx[2*i+1] = i+1
rx = ceed.ElemRestriction(nelem, 2, nx, 1, indx, cmode=libceed.USE_POINTER)

indu = np.zeros(nelem*p, dtype="int32")
for i in range(nelem):
  for j in range(p):
    indu[p*i+j] = i*(p-1) + j
ru = ceed.ElemRestriction(nelem, p, nu, 1, indu, cmode=libceed.USE_POINTER)
strides = np.array([1, q, q], dtype="int32")
rui = ceed.StridedElemRestriction(nelem, q, q*nelem, 1, strides)

# Bases
bx = ceed.BasisTensorH1Lagrange(1, 1, 2, q, libceed.GAUSS)
bu = ceed.BasisTensorH1Lagrange(1, 1, p, q, libceed.GAUSS)

# QFunctions
qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

# Operators
op_setup = ceed.Operator(qf_setup)
op_setup.set_field("dx", rx, bx, libceed.VECTOR_ACTIVE)
op_setup.set_field("weights", libceed.ELEMRESTRICTION_NONE, bx,
                   libceed.VECTOR_NONE)
op_setup.set_field("qdata", rui, libceed.BASIS_COLLOCATED,
                   libceed.VECTOR_ACTIVE)

op_mass = ceed.Operator(qf_mass)
op_mass.set_field("u", ru, bu, libceed.VECTOR_ACTIVE)
op_mass.set_field("qdata", rui, libceed.BASIS_COLLOCATED, qdata)
op_mass.set_field("v", ru, bu, libceed.VECTOR_ACTIVE)

# Setup
op_setup.apply(x, qdata)

# Apply mass matrix
u.set_value(0)
op_mass.apply(u, v)

# Check
v_array = v.get_array_read()

print(v_array)

v.restore_array_read()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
